In [ ]:
### TO DO: finish different types of analysis

In [1]:
### Given dictionary of changes, analyze the changes

In [30]:
import databaseOps, fields, utility
import numpy as np
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup, Tag
import copy
from datetime import datetime as dt

In [28]:
import json
from pprint import pprint

#with open('changes.json') as data_file:    
with open('2016Q4_2017Q4_3MCOChanges.json') as data_file:
    jsonChanges = json.load(data_file)

In [3]:
numerical = ["transactionShares", "transactionPricePerShare", "sharesOwnedFollowingTransaction", "conversionOrExercisePrice", "underlyingSecurityShares"]

categorical = ["securityTitle", "transactionCode", "equitySwapInvolved", "transactionAcquiredDisposedCode", "directOrIndirectOwnership", "natureOfOwnership", "transactionTimelines", "underlyingSecurityTitlesecurityTitle"]

# categorical data w specified categories
typedCategorical = ["transactionCode", "equitySwapInvolved", "transactionAcquiredDisposedCode", "directOrIndirectOwnership", "transactionTimelines"]

date = ["transactionDate", "exerciseDate", "expirationDate"]

formPortion = ["head", "dt", "ndt"]

In [17]:
### Functions to perform basic analysis on dictionary of changes

def getCatAnalysis(changedFieldName, beforeVal, afterVal):
    catAnalysis = dict()
    catAnalysis["dataType"] = "categorical"
    catAnalysis["fieldName"] = changedFieldName
    
    if beforeVal == '':
        catAnalysis["changeType"] = "insert"
    elif afterVal == "":
        catAnalysis["changeType"] = "delete"
    else:
        catAnalysis["changeType"] = "update"
        
    if changedFieldName in typedCategorical:
        catAnalysis["updateVector"] = (beforeVal, afterVal)
    else:
        catAnalysis["editDistance"] = utility.editDistance(beforeVal, afterVal)
    
    return catAnalysis

def getNumAnalysis(changedFieldName, beforeVal, afterVal):
    numAnalysis = dict()
    numAnalysis["fieldName"] = changedFieldName
    numAnalysis["dataType"] = "numerical"
    
    if beforeVal == '':
        beforeVal = 0
        numAnalysis["changeType"] = "insert"
        afterVal = float(afterVal)
    elif afterVal == '':
        afterVal = 0
        numAnalysis["changeType"] = "delete"
        beforeVal = float(beforeVal)
        
    if beforeVal != "" and afterVal != "":
        beforeVal, afterVal = float(beforeVal), float(afterVal)
    
    change = afterVal - beforeVal
    if change != 0:
        numAnalysis["amountChanged"] = change        
        if afterVal > beforeVal:
            numAnalysis["changeDirection"] = "+" 
        else:
            numAnalysis["changeDirection"] = "-" 
        numAnalysis["percentChange"] = round(abs(change)/beforeVal*100, 3)      
        numAnalysis["changeType"] = "update"
    return numAnalysis

def getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent):
    if len(beforeFoot) != 0 or len(afterFoot) != 0:
        if len(beforeFoot) > len(afterFoot):
            footChangeType = "footnote in 4"
        elif len(beforeFoot) < len(afterFoot):
            footChangeType = "footnote in 4/a"
        else:
            diffCount = sum([1 for i in range(len(beforeFoot)) if bFootContent[i] != aFootContent[i]])
            if diffCount != 0:
                footChangeType = "footnote content diff"
            else:
                if beforeFoot != afterFoot:
                    footChangeType = "footnote id diff"
                else:
                    footChangeType = ""
        if footChangeType != "":
            if "changeType" in analysisDic:
                analysisDic["changeType"] = [analysisDic["changeType"], footChangeType]
            else:
                analysisDic["changeType"] = footChangeType
    return analysisDic

def getDateAnalysis(changedFieldName, beforeVal, afterVala):
    dateAnalysis = dict()
    dateAnalysis["dataType"] = "date"
    if beforeVal == "":
        dateAnalysis["changeType"] = "insert"
    if afterVal == "":
        dateAnalysis["changeType"] = "delete"
    if beforeVal != "" and afterVal != "":
        beforeDate = dt.strptime(beforeVal, "%Y-%m-%d")
        afterDate = dt.strptime(afterVal, "%Y-%m-%d")
        dateDiff = beforeDate - afterDate
        if dateDiff != 0:
            dateAnalysis["changeType"] = "update"
            if beforeDate > afterDate:
                dateAnalysis["changeDirection"] = "+" #changed date to later
            elif beforeDate < afterDate:
                dateAnalysis["changeDirection"] = "-"
            dateAnalysis["amountChanged"] = dateDiff.total_seconds()/60/60/24 #days
    return dateAnalysis


def runAnalysis(fileDic):
    files = copy.deepcopy(fileDic)
    for file in files:
        changeDic = files[file]
        for portion in formPortion:
            for rowKey in list(changeDic[portion].keys()):
                row = changeDic[portion][rowKey]
                
                for changedFieldName in list(row.keys()):
                    footnoteChangeOnly = False

                    if "value" in row[changedFieldName]['4']:
                        beforeVal = row[changedFieldName]['4']['value']
                    else:
                        beforeVal = ""
                    if "value" in row[changedFieldName]['4A']:
                        afterVal = row[changedFieldName]['4A']['value'] 
                    else: 
                        afterVal = ""

                    beforeFoot, afterFoot = None, None
                    if isinstance(beforeVal, str) and ("<value>" in beforeVal or "footnoteId" in beforeVal):
                        beforeSoup = BeautifulSoup(beforeVal)
                        beforeVal = ''.join([str(val.text) for val in beforeSoup.findAll('value')])
                        beforeFoot = [footnote.get('id') for footnote in beforeSoup.findAll('footnoteid')] #soup made it lowercased
                        if beforeVal == '':
                            footnoteChangeOnly = True

                    if isinstance(afterVal, str) and ("<value>" in afterVal or "footnoteId" in afterVal):
                        afterSoup = BeautifulSoup(afterVal)
                        afterVal = ''.join([str(val.text) for val in afterSoup.findAll('value')])
                        afterFoot = [footnote.get('id') for footnote in afterSoup.findAll('footnoteid')]

                        if footnoteChangeOnly:
                            if afterVal != '':
                                footnoteChangeOnly = False

                    bFootContent = [row[changedFieldName]['4'][fid] for fid in beforeFoot]
                    aFootContent = [row[changedFieldName]['4A'][fid] for fid in afterFoot]
                        
                    if footnoteChangeOnly:
                        row[changedFieldName]["analysis"] = getFootnoteAnalysis(dict(), beforeFoot, afterFoot, bFootContent, aFootContent)
                    else: 
                        if changedFieldName in categorical:
                            analysisDic = getCatAnalysis(changedFieldName, beforeVal, afterVal)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)
                        elif changedFieldName in numerical:
                            #if "footnoteId" not in beforeVal and "footnoteId" not in afterVal:
                            analysisDic = getNumAnalysis(changedFieldName, beforeVal, afterVal)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)
                        elif changedFieldName in date:
                            analysisDic = getDateAnalysis(changedFieldName, beforeVal, afterVal, beforeFoot, afterFoot, bFootContent, aFootContent)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)


    print(json.dumps(files, indent=2))
    return files

In [19]:
analysis = runAnalysis(jsonChanges)

{
  "66740/0001127602-17-007798": {
    "dt": {
      "1-transaction": {
        "securityTitle": {
          "4A": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value>"
          },
          "4": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value><footnoteId id=\"F4\"/>",
            "F4": "This option becomes exercisable in equal installments on each of the first three anniversaries of the grant date (2/7/2017)."
          },
          "analysis": {
            "fieldName": "securityTitle",
            "dataType": "categorical",
            "changeType": "footnote in 4"
          }
        }
      }
    },
    "head": {},
    "ndt": {
      "1-holding": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>805</value>"
          },
          "4": {
            "value": "<value>805</value><footnoteId id=\"F3\"/>",
            "F3": "Includes shares acquired pursuant to the 3M Voluntary In

In [6]:
### Functions to summarize changes & analyses 

def getAllAnalyses(files, dataType, analysisType=False, analysisSubType=False):
    analyses = {}
    analyses["data"] = []
    if analysisType in categoricalAnalysisType:    
        analyses["metadata"] = {}
        for option in categoricalAnalysisType[analysisType]:
            analyses["metadata"][option] = 0
        
    for file in files:
        changeDic = files[file]
        for portion in formPortion:
            for rowKey in list(changeDic[portion].keys()):
                row = changeDic[portion][rowKey]
                for changedFieldName in list(row.keys()):
                    if "analysis" in row[changedFieldName] and dataType == row[changedFieldName]["analysis"]["dataType"]:
                        if analysisType and analysisType in row[changedFieldName]["analysis"]:
                            data = row[changedFieldName]["analysis"][analysisType]
                            
                            if analysisType in categoricalAnalysisType:
                                analyses["metadata"][data] += 1
                            analyses["data"].append(data)
                        elif not analysisType:
                            analyses["data"].append(row[changedFieldName]["analysis"])
                            
#     if analysisType in numericalAnalysisType:
#         analyses["metadata"] = getNumMetadata(analyses["data"])
    
    return analyses

In [25]:
print(getAllAnalyses(analysis, "numerical", "amountChanged"))
print(getAllAnalyses(analysis, "numerical", "changeDirection"))

{'data': [8358.3881, -18.0, 4312.3881, 6763.730299999999, -14.0, 3482.7302999999993, 4159.0412, -12.0, 2131.0411999999997, 8358.0, 4294.0, 7455.842700000001, 69.15799999999999, 7176.842700000001, 10702.0, 5508.0, 8359.0, 4295.0, 8358.0, 4294.0, 6752.2304, 59.69200000000001, 6499.2304, 8359.0383, 80.94399999999999, 8047.0383, 1026.0, 531.0, 4159.0, 2119.0, 4763.0, 2432.0, 6764.0, 3469.0, 47663.6563, 914.2860000000001, 45594.6563, -11.0, 11.0, 8358.7475, -17.0, 4311.7474999999995, 1663.0, 827.0, 8359.0, 4295.0, 10702.0213, -18.0, 5526.0213, 1662.7466999999997, -12.0, 838.7467000000001, 6752.0, 6559.0, 5952.0, 5777.0, 1025.8474999999999, 26.0, 504.84749999999985, 7456.0, 7246.0, 47664.0, 46509.0, 8358.7782, -17.0, 4311.778200000001, 5951.849900000001, 39.51400000000001, 5737.849900000001, 8359.0, 8128.0, -2.0, 8358.497600000002, -20.0, 4314.497600000002, 4763.0262, -13.0, 2445.0262]}
{'data': ['+', '-', '+', '+', '-', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '

In [27]:
print(json.dumps(getAllAnalyses(analysis, "numerical"), indent=2))

{
  "data": [
    {
      "dataType": "numerical",
      "fieldName": "sharesOwnedFollowingTransaction",
      "changeType": "footnote in 4"
    },
    {
      "dataType": "numerical",
      "fieldName": "transactionShares",
      "changeType": "footnote in 4"
    },
    {
      "dataType": "numerical",
      "changeType": "update",
      "amountChanged": 8358.3881,
      "percentChange": 37.433,
      "fieldName": "sharesOwnedFollowingTransaction",
      "changeDirection": "+"
    },
    {
      "dataType": "numerical",
      "changeType": [
        "update",
        "footnote content diff"
      ],
      "amountChanged": -18.0,
      "percentChange": 0.443,
      "fieldName": "transactionShares",
      "changeDirection": "-"
    },
    {
      "dataType": "numerical",
      "changeType": [
        "update",
        "footnote in 4"
      ],
      "amountChanged": 4312.3881,
      "percentChange": 19.313,
      "fieldName": "sharesOwnedFollowingTransaction",
      "changeDirection": "+

In [21]:
dataType = ["categorical", "numerical", "footnote", "date"]
categoricalAnalysisType = {
    "changeDirection": ["+", "=", "-"],
    "changeType": ["noChange", "footnote", "madeEarlier", "madeLater", "update"]
}

numericalAnalysisType = ["amountChanged", "percentChange"]

def getNumChanges(files, dataType, analysisType=False, analysisSubType=False):
    analyses = getAllAnalyses(files, dataType, analysisType)
    if analysisType is False:
        print("Number of {} changes: {}".format(dataType, len(analyses["data"])))
    if analysisType is not False:
        print("Number of {} changes in {}: {}".format(dataType, analysisType, len(analyses["data"])))
        if analysisType in categoricalAnalysisType:
            options = categoricalAnalysisType[analysisType]
            for option in options:
                print("Number of {} changes with {} of {}: {}".format(dataType, analysisType, option, analyses["metadata"][option]))
            
# az = [getNumChanges(analysis, i) for i in dataType]
# getNumChanges(analysis, "numerical", "amountChanged")
# getNumChanges(analysis, "numerical", "changeDirection")

Number of numerical changes in amountChanged: 78
Number of numerical changes in changeDirection: 78
Number of numerical changes with changeDirection of +: 67
Number of numerical changes with changeDirection of =: 0
Number of numerical changes with changeDirection of -: 11


In [22]:
# def getCatStats(dic):

def getNumStats(data, dataType, analysisType): #for distributions
    print("Mean", np.mean(data))
    print("Median", np.median(data))
    print("25% Percentile", np.percentile(data, 25))
    print("75% Percentile", np.percentile(data, 75))
    print("Standard deviation", np.std(data))
    hist, bins = np.histogram(data)
    plt.hist(data, bins)
    plt.title("Distribution of {} changes in {}".format(dataType, analysisType))
    plt.show()

def getNumMetadata(files, dataType, analysisType=False, analysisSubType=False):
    data = getAllAnalyses(files, dataType, analysisType)["data"]
    getStats(data)
    
# getNumMetadata(analysis, "numerical", "amountChanged")

NameError: name 'getStats' is not defined

In [ ]:
realChanges = {
  "1545654/000122520817001902/0001225208-17-001902": {
    "dt": {},
    "accNum": "1545654/0001225208-17-001730",
    "head": {},
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4A": {
            "value": "<value>615.0000</value><footnoteId id=\"F2\"/>",
            "F2": "The number of shares reflected in the Reporting Person's Form 4 (filed on January 26, 2017) was incorrectly reported. The correct number of shares is reflected in this amendment."
          },
          "4": {
            "value": "<value>678.0000</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>79801.6695</value>"
          },
          "4": {
            "value": "<value>79738.6695</value>"
          }
        }
      }
    }
  },
  "1253352/000114036117015044/0001140361-17-015044": {
    "dt": {
      "2-transaction": {
        "expirationDate": {
          "4A": {
            "F4": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date.",
            "value": "<footnoteId id=\"F4\"/>"
          },
          "4": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date."
          }
        },
        "exerciseDate": {
          "4A": {
            "F4": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date.",
            "value": "<footnoteId id=\"F4\"/>"
          },
          "4": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date."
          }
        }
      },
      "1-transaction": {
        "expirationDate": {
          "4A": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          },
          "4": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          }
        },
        "conversionOrExercisePrice": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        },
        "exerciseDate": {
          "4A": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          },
          "4": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          }
        }
      }
    },
    "accNum": "1253352/0001140361-17-001007",
    "head": {},
    "ndt": {
      "2-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>10960</value>"
          },
          "4": {
            "value": "<value>10340</value>"
          }
        }
      },
      "1-transaction": {
        "transactionShares": {
          "4A": {
            "F1": "This amendment is being filed to remove reference to improperly withheld shares of common stock issued to the reporting person pursuant to the vesting of restricted stock units. Due to an administrative error, upon the vesting of restricted stock units on 1/4/2017, 620 shares of common stock were withheld to satisfy tax withholding obligations as if the reporting person were an employee of the issuer. This error was corrected and the improperly-withheld 620 shares of common stock were issued to the reporting person on 4/3/2017.",
            "value": "<value>1898</value><footnoteId id=\"F1\"/>"
          },
          "4": {
            "value": "<value>1898</value>"
          }
        },
        "transactionPricePerShare": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        }
      }
    }
  },
  "1444236/000120919117016922/0001209191-17-016922": {
    "dt": {
      "1-transaction": {
        "exerciseDate": {
          "4A": {
            "value": "<value>2018-02-23</value><footnoteId id=\"F3\"/>",
            "F3": "This option vests in four equal annual installments beginning February 23, 2018."
          },
          "4": {
            "value": "<value>2018-02-23</value><footnoteId id=\"F2\"/>",
            "F2": "This option vests in four equal annual installments beginning February 23, 2018."
          }
        }
      }
    },
    "accNum": "1444236/0001209191-17-014948",
    "head": {},
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>17625</value><footnoteId id=\"F2\"/>",
            "F2": "This Amendment is being filed to correct an error in the number of shares of Series A Common Stock beneficially owned after the reported transaction."
          },
          "4": {
            "value": "<value>45833</value>"
          }
        }
      }
    }
  }
}

sampleChanges = {
    "num": {
    "dt": {
      "1-transaction": {
        "securityTitle": {
          "4A": {
            "value": "some title"
          },
          "4": {
            "value": "new title"
          }
        }, "directOrIndirectOwnership": {
          "4A": {
            "value": "<value>D</value>"
          },
          "4": {
            "value": "<value>I</value>"
          }
        },
        "conversionOrExercisePrice": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        }
      }
    },
    "accNum": "1444236/0001209191-17-014948",
    "head": {},
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>17625</value><footnoteId id=\"F2\"/>",
            "F2": "This Amendment is being filed to correct an error in the number of shares of Series A Common Stock beneficially owned after the reported transaction."
          },
          "4": {
            "value": "<value>45833</value>"
          }
        }, "conversionOrExercisePrice": {
          "4A": {
            "value": "32323"
          },
          "4": {
            "value": "2"
          }
        }
      }
    }
}    
}

In [3]:
import sys
!{sys.executable} -m pip install pandas

    100% |████████████████████████████████| 14.4MB 788kB/s  a 0:00:011
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
